In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q keras
import pandas as pd

In [3]:
df=pd.read_csv("/content/drive/MyDrive/ann/Churn_Modelling.csv")
X=df.iloc[:,3:13]
y=df.iloc[:,13]

In [4]:
#creating dummy variables of categorical features
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [5]:
X=pd.concat([X,geography,gender],axis=1) #concatinating
X=X.drop(['Geography','Gender'],axis=1) #droping the unnecessary columns

In [6]:
#splitting the dataset in to train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
#now lets make ANN
import keras
from keras.models import Sequential#sequential library is responsible for creating neural networks
from keras.layers import Dense, Activation, Embedding, Flatten,LeakyReLU,BatchNormalization #dense library is used to create the hidden layers
from keras.layers import Dropout #dropout is an regularization library for neural networks
from keras.activations import relu,sigmoid

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [24]:

def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0: #for the 1st input and hidden layer
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation)) #choosing the activation function
            model.add(Dropout(0.3)) #dropuot rate
        else:
            model.add(Dense(nodes)) #for the hidden layers
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    #if we are using relu activation he_uniform and he_normal is better
    #if we are using sigmoid glorot is better
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)


layers = [[20], [40, 20], [45, 30, 15]] #no.of layers with number of neurons
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


In [25]:
grid_result=grid.fit(X_train,y_train)

In [26]:
print(grid_result.best_score_,grid_result.best_params_)

0.8558750152587891 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [29]:
pred_y=grid.predict(X_test)
y_pred=(pred_y > 0.5)

In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_pred,y_test)
print(cm)

[[1513  193]
 [  82  212]]


In [32]:
score=accuracy_score(y_pred,y_test)

In [33]:
print(score)

0.8625
